In [1]:
import os
import cv2
import numpy as np
from glob import glob
from tqdm import tqdm
import warnings

In [2]:
base_path = "/sfs/gpfs/tardis/home/kcm7zp/brain_tumor_data"

tumor_types = ['glioma', 'meningioma', 'pituitary']
label_map = {'glioma': 1, 'meningioma': 2, 'pituitary': 3}

# print(os.listdir(base_path))

In [3]:
data = []

for tumor in tumor_types:
    img_folder = os.path.join(base_path, tumor, "images")
    mask_folder = os.path.join(base_path, tumor, "masks")
    
    img_files = sorted([f for f in os.listdir(img_folder) if f.endswith(".png") and not f.startswith(".")])
    
    for img_file in tqdm(img_files, desc=f"Processing {tumor}"):
        base_id = img_file.replace(f"{tumor}_", "").replace(".png", "")
        mask_file = f"{tumor}_mask_{base_id}.png"
        
        img_path = os.path.join(img_folder, img_file)
        mask_path = os.path.join(mask_folder, mask_file)
        
        # Read in the image and mask 
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        if img is None or mask is None:
            print(f"Failed to load: {img_path} or {mask_path}")
            continue

        # Resize to common dimensions
        img = cv2.resize(img, (256, 256))
        mask = cv2.resize(mask, (256, 256))

        # Normalize to [0, 1]
        img = img / 255.0
        mask = mask / 255.0
        
        # Append to data list
        data.append({
            'image': img,
            'mask': mask,
            'label': label_map[tumor]
        })

print(f"\nTotal samples loaded: {len(data)}")


Processing pituitary: 100%|██████████| 930/930 [00:11<00:00, 80.49it/s] 


Total samples loaded: 3064


In [4]:
# Sanity check 
print(f"Total samples loaded: {len(data)}")

labels = [item['label'] for item in data]
unique_labels, counts = np.unique(labels, return_counts=True)
print("Labels and counts:", dict(zip(unique_labels, counts)))

Total samples loaded: 3064
Labels and counts: {1: 1426, 2: 708, 3: 930}


In [5]:
# Convert to numpy arrays -- run this again
# Skip converting masks right now to save memory
images = np.array([item['image'] for item in data])
# masks = np.array([item['mask'] for item in data])
labels = np.array([item['label'] for item in data])

# Add channel dimension for Tensorflow/Keras 
images = np.expand_dims(images, axis=-1)  # shape: (N, 256, 256, 1)
# masks = np.expand_dims(masks, axis=-1)

In [6]:
# Train/Val/Test Split 
from sklearn.model_selection import train_test_split

# First split off test
X_trainval, X_test, y_trainval, y_test = train_test_split(images, labels, test_size=0.15, stratify=labels, random_state=42)

# Then split train and val
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.15, stratify=y_trainval, random_state=42)

## Build the CNN in Keras

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.metrics import Precision, Recall, SparseCategoricalAccuracy

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 tumor classes
])

2025-04-17 11:49:24.903952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-17 11:49:25.089195: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8473] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-17 11:49:25.137602: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1471] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-17 11:49:25.517363: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-17 11:49:29.922634: I tensorflo

## Compile the Model

In [8]:
# Shift labels to 0-based indexing (1 --> 0, etc.) 
y_train = y_train - 1
y_val = y_val - 1
y_test = y_test - 1

y_train = y_train.flatten()
y_val = y_val.flatten()
y_test = y_test.flatten()


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## Train the Model

In [9]:
print(X_train.shape) 
print(y_train.shape)
print(np.unique(y_train, return_counts=True))

## Output
# (2213, 256, 256, 1)
# (2213,)
# (array([0, 1, 2]), array([count_0, count_1, count_2]))

(2213, 256, 256, 1)
(2213,)
(array([0, 1, 2]), array([1030,  512,  671]))


In [10]:
# Check for NaNs or Inf values
print(np.isnan(X_train).sum(), np.isinf(X_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())

0 0
0 0


In [11]:
print(np.unique(y_train))     # Should output [0 1 2]
print(y_train.shape)          # Should be (N,) — a 1D array
print(y_train[:5])            # Check first few labels


[0 1 2]
(2213,)
[2 1 0 2 2]


In [ ]:
print(images.shape)           # (3064, 256, 256, 1)
print(labels.shape)           # (3064,)
print(labels.dtype)           # should be int32 or int64

In [12]:
# Suppress warnings

import os
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore")
logging.getLogger("tensorflow").setLevel(logging.ERROR)


In [13]:
# Hide warnings 
warnings.filterwarnings("ignore")

# Flatten labels before fitting 
# y_train = y_train.flatten()
# y_val = y_val.flatten()
# y_test = y_test.flatten()

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val)
)

Epoch 1/10


2025-04-17 11:49:51.333198: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90501
W0000 00:00:1744904991.629366  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904991.661352  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904991.683678  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904991.684157  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904991.685041  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904991.685671  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904991.686740  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904991.687769  240724 gpu_

 1/70 [..............................] - ETA: 4:20 - loss: 1.0898 - accuracy: 0.5000

W0000 00:00:1744904994.098173  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904994.098626  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904994.099373  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904994.099771  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904994.100511  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904994.101040  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904994.101960  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904994.102842  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744904994.103840  240726 gp

 2/70 [..............................] - ETA: 27s - loss: 2.3940 - accuracy: 0.5156 

...

70/70 [==============================] - 1s 17ms/step - loss: 0.0395 - accuracy: 0.9878 - val_loss: 0.2025 - val_accuracy: 0.9156
Epoch 6/10
 4/70 [>.............................] - ETA: 1s - loss: 0.0124 - accuracy: 1.0000

W0000 00:00:1744905001.101477  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905001.101712  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905001.101918  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905001.102195  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905001.102466  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905001.102686  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905001.102862  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905001.103068  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905001.103359  240726 gp

70/70 [==============================] - 1s 17ms/step - loss: 0.0139 - accuracy: 0.9973 - val_loss: 0.2262 - val_accuracy: 0.9258
Epoch 7/10
 5/70 [=>............................] - ETA: 1s - loss: 0.0062 - accuracy: 1.0000

W0000 00:00:1744905002.314687  240727 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905002.314921  240727 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905002.315127  240727 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905002.315404  240727 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905002.315674  240727 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905002.315896  240727 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905002.316072  240727 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905002.316281  240727 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905002.316582  240727 gp

70/70 [==============================] - 1s 17ms/step - loss: 0.0096 - accuracy: 0.9973 - val_loss: 0.3723 - val_accuracy: 0.9130
Epoch 8/10
70/70 [==============================] - 1s 17ms/step - loss: 0.0071 - accuracy: 0.9986 - val_loss: 0.3552 - val_accuracy: 0.9309
Epoch 9/10
70/70 [==============================] - 1s 17ms/step - loss: 0.0069 - accuracy: 0.9977 - val_loss: 0.3235 - val_accuracy: 0.9028
Epoch 10/10
70/70 [==============================] - 1s 17ms/step - loss: 0.0203 - accuracy: 0.9928 - val_loss: 0.3173 - val_accuracy: 0.9156


## Evaluate the Model

In [14]:
from sklearn.metrics import classification_report
import numpy as np

test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {test_acc:.2f}")

# Get predictions
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Flatten true labels if necessary
y_true = y_test.flatten()

# Print classification report
print(classification_report(y_true, y_pred, target_names=['Meningioma', 'Glioma', 'Pituitary']))

15/15 [==============================] - 0s 9ms/step - loss: 0.4460 - accuracy: 0.9130

Test Accuracy: 0.91


W0000 00:00:1744905051.979306  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905051.979529  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905051.979875  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905051.980059  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905051.980402  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905051.980657  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905051.981067  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905051.981460  240726 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905051.981918  240726 gp

15/15 [==============================] - 0s 8ms/step
              precision    recall  f1-score   support

  Meningioma       0.96      0.88      0.92       214
      Glioma       0.79      0.87      0.83       106
   Pituitary       0.95      0.99      0.97       140

    accuracy                           0.91       460
   macro avg       0.90      0.91      0.91       460
weighted avg       0.92      0.91      0.91       460



W0000 00:00:1744905052.339087  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905052.339310  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905052.339614  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905052.339791  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905052.340092  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905052.340320  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905052.340689  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905052.341046  240724 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1744905052.341451  240724 gp